In [3]:
from PIL import Image
import pytesseract

In [6]:
print(pytesseract.image_to_string(Image.open('images/test.png')))


This is a lot of 12 point text to test the
ocr code and see if it works on all types
of file format.

The quick brown dog jumped over the
lazy fox. The quick brown dog jumped
over the lazy fox. The quick brown dog
jumped over the lazy fox. The quick
brown dog jumped over the lazy fox.

